In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

import matplotlib.pyplot as plt
%matplotlib inline

# 1 - Un exemple simple

#### Définition de la fonction

Nous allons utiliser un réseau de neurone pour simuler la fonction:

$f(x)=\begin{cases}
    0, & \text{si $\, 0 \leq x \leq 0.5$}\\
    1, & \text{si $\, 0.5 < x \leq 1$}
  \end{cases}$

In [ ]:
def f(x):
    return (x > 0.5)

In [ ]:
x = torch.Tensor([0, 0.2, 0.4, 0.6, 0.8, 1])
y = f(x)

print("x    :", x)
print("f(x) :", y)

#### Construction du dataset

On construit un dataset de taille 1000 contenant des nombres aléatoires entre 0 et 1.  

Ensuite, on divise ce dataset en 2 parties:
- 80% seront des exemples d'entrainement (utilisés pour entraîner notre réseau)
- 20% seront des exemples de validation (utilisés pour tester la qualité de notre réseau)

Il est important de tester notre réseau sur des exemples qu'il n'a pas vu lors de son entrainement.

In [ ]:
dataset_size = 1000
train_size = dataset_size * 8 // 10

numbers = torch.rand(dataset_size)  # On créé 1000 nombres aléatoires entre 0 et 1
target = f(numbers).long()  # On applique la fonction f à ces nombres

train_dataset = TensorDataset(numbers[:train_size], target[:train_size])
val_dataset = TensorDataset(numbers[train_size:], target[train_size:])

print(train_dataset[:5])

#### Définition du modèle du réseau de neurones

On définit un réseau de neurones avec **une couche cachée de taille 5**.  
On utilise la non-linéarité ReLU.

In [ ]:
model = nn.Sequential(
    nn.Linear(1, 5),  # On met 1 car en entrée il n'y a qu'un seul nombre x
    nn.ReLU(),
    nn.Linear(5, 2),  # On met 2 car en sortie on prédit 2 classes 0 ou 1
)
print(model)

#### Entrainement du réseau

Pour entrainer notre modèle, il faut définir:
- Un "optimiser" : C'est lui qui va **mettre à jour les paramètres du modèle** (en utilisant la méthode de descente de gradient).
- Une fonction de loss : C'est la fonction qui **mesure la qualité de la prédiction** de notre réseau.  
Plus la valeur de cette fonction est basse, plus la prediction est bonne. C'est donc cette fonction qu'on va minimiser avec notre optimiser.

In [ ]:
batch_size = 10  # Nombre d'exemples d'entrainement entre chaque mise à jour des paramètres
nb_epochs = 2   # Nombre de fois qu'on voit tous les exemples d'entrainement
lr = .5  # "Learning Rate" : valeur qui multiplie le gradient dans la méthode de "descente de gradient"

dataloader = DataLoader(train_dataset, batch_size)  # Le Dataloader sert à créer des groupes d'exemples d'entrainement (appelés batchs)
optimizer = optim.SGD(model.parameters(), lr)  # L'optimiser qu'on utilise s'appelle SGD : Stochastic Gradient Descent
loss_function = nn.NLLLoss()  # Cette fonction de loss calcule le log de la probabilité 

for epoch in range(nb_epochs):
    for inputs, targets in dataloader:
        optimizer.zero_grad()
        outputs = model(inputs.view(-1, 1))
        probabilities = outputs.softmax(1) # Le softmax transforme la sortie du modèle en probabilités qui somment à 1
        loss = loss_function(probabilities, targets)
        loss.backward()
        optimizer.step()

Notre réseau est maintenant entrainé !

#### Test du réseau (Quantitatif)

On teste notre modèle en calculant la précision de la prédiction sur notre ensemble de validation.  

La précision est définit comme :  $\text{précision} = \dfrac{\text{Nombre d'exemples correctement classifiés}}{\text{Nombre d'exemples total}}$

In [ ]:
dataloader = DataLoader(val_dataset, batch_size)  # On utilise le dataset de validation

acc = 0
for inputs, targets in dataloader:
    outputs = model(inputs.view(-1, 1))
    predictions = outputs.argmax(1)
    acc += (predictions == targets).sum()

acc = acc.float() / len(dataloader.dataset)
print("Précision: {:.2%}".format(acc))

#### Test du réseau (Qualitatif)

In [ ]:
# Testez votre réseau !

number = 0.45
output = model(torch.Tensor([[number]]))
print(output.softmax(1))  # Le softmax transforme la sortie du modèle en probabilités dont la somme fait 1

On va afficher la **fonction de décision** que le réseau a appris  :
- En bleue, la fonction qui associe à chaque $x$ la probabilité que $x$ est au dessus de 0.5 
- En orange, la décision prise : si la probabilité est inférieure à 0.5, on prédit 0, sinon on prédit 1.

In [ ]:
with torch.no_grad():
    x = torch.linspace(0,1)  # On créé une liste de nombres qui vont de 0 à 1 de facon régulière
    y = model(x.unsqueeze(1)).softmax(1)[:, 1]  # On calcule la probabilité (courbe bleue) avec le réseau
    decision = y.round()  # On calcule la décision (courbe orange) en arrondissant la probabilité

plt.figure(figsize=(14,7))
plt.plot(x.numpy(), y.numpy(), color='#1f77b4')
plt.plot(x.numpy(), decision.numpy(), color='#ff7f0e');

#### Améliorer les résultats

Pour améliorer les résultats, essayer d'augmenter le nombre d'époques.  
Par exemple, changer de 2 à 10 puis relancer la fonction précedente pour voir quelle fonction le réseau a appris.

Dans ce cas, cela devrait suffire. En règle générale, on peut essayer de changer de nombreux paramètres:
- Learning Rate
- Batch Size
- Paramètres de l'optimiser
- Modèles (Mettre plus de neurones, plus de couches, ...)

#### Regardez les poids appris

In [ ]:
model[0].weight

# 2 - Un autre exemple

### A vous d'essayer !

Maintenant, nous allons essayer de simuler la fonction:

$f(x)=\begin{cases}
    1, & \text{si $ \, x < 0.1$ ou $ \, 0.5 < x < 0.8$}\\
    0, & \text{sinon}
  \end{cases}$
  
Ci-dessous, vous pouvez utiliser:
- le symbole `&` pour comme condition "et"
- le symbole `|` comme condition "ou"

Par exemple: ```(x > 0) & (x < 1)```


In [ ]:
# %load sol_f

def f(x):
    return # Ecrire ici

On construit un dataset :

In [ ]:
dataset_size = 10000
train_size = dataset_size * 8 // 10

numbers = torch.rand(dataset_size)  # On créé 1000 nombres aléatoires entre 0 et 1
target = f(numbers).long()  # On applique la fonction f à ces nombres

train_dataset = TensorDataset(numbers[:train_size], target[:train_size])
val_dataset = TensorDataset(numbers[train_size:], target[train_size:])

print(train_dataset[:5])

On construit le modèle :  
$\Longrightarrow$ Contruisez un modèle avec 2 couches cachées de 10 et 15 neurones respectivement.

In [ ]:
# %load sol_model_2

model = nn.Sequential(
    # Ecrire ici
)

print(model)

On entraine le réseau :

In [ ]:
batch_size = 50
nb_epochs = 10
lr = 0.5

dataloader = DataLoader(train_dataset, batch_size)
optimizer = optim.SGD(model.parameters(), lr)
loss_function = nn.CrossEntropyLoss()  # Cette fonction de loss combine le softmax et le calcul du log de la probabilité 

for epoch in range(nb_epochs):
    for inputs, targets in dataloader:
        optimizer.zero_grad()
        outputs = model(inputs.view(-1, 1))
        loss = loss_function(outputs, targets)
        loss.backward()
        optimizer.step()

On teste le réseau :

In [ ]:
dataloader = DataLoader(val_dataset, batch_size)

acc = 0
for inputs, targets in dataloader:
    outputs = model(inputs.view(-1, 1))
    predictions = outputs.argmax(1)
    acc += (predictions == targets).sum()

acc = acc.float() / len(dataloader.dataset)
print("Précision: {:.2%}".format(acc))

On regarde la fonction de décision :

In [ ]:
with torch.no_grad():
    x = torch.linspace(0,1)
    y = model(x.unsqueeze(1)).softmax(1)[:, 1]
    decision = y.round()

plt.figure(figsize=(14,7))
plt.plot(x.numpy(), y.numpy(), color='#1f77b4')
plt.plot(x.numpy(), decision.numpy(), color='#ff7f0e');

# 3 - Un réseau utile : Classification d'images de nombre

### Dataset

Nous allons utiliser un réseau de neurones pour résoudre un problème de classification d'images de chiffres manuscrits.  
Le dataset que nous allons utiliser s'appelle MNIST.

In [ ]:
import torchvision.datasets as datasets
import torchvision.transforms as transforms

# MNIST Dataset (Images and Labels)
train_dataset = datasets.MNIST(root='./data', train=True, transform=transforms.ToTensor(), download=True)
val_dataset = datasets.MNIST(root='./data', train=False, transform=transforms.ToTensor(), download=True)

print("Taille du dataset d'entrainement : %d images" % len(train_dataset))
print("Taille du dataset de validation : %d images" % len(val_dataset))

**Regardons des exemples d'images:**

In [ ]:
fig = plt.figure(figsize=(16,9))
for i in range(0, 5):
    data, label = val_dataset[i]
    data = data.squeeze()
    fig.add_subplot(1, 5, i+1)
    plt.imshow(data, cmap="gray")
    plt.xlabel(label, fontsize=20)

**Combien de classes le dataset contient-il ?**

In [ ]:
input("Nombre de classes :");

**Si on choisir au hasard, quelle est la probabilité qu'on choisisse la bonne classe ? Comment vérifier cette hypothèse ?**

In [ ]:
input("Probabilité de choisir la bonne classe au hasard :");

### Modèle

On construit un modèle de réseau de neurones pour classifier les images.
- En entrée, on a des images noir et blanc de taille 28x28 = 784 pixels.  
On va "simplement" transformer chaque image en un grand vecteur de 784 nombres, on a donc **784 valeurs** en entrée.
- On va utiliser 2 couches cachées de **75 et 50 neurones** respectivement.
- En sortie, on veut la probabilité pour chaque classe, il faut donc **autant de neurones que de classes** en sortie.

In [ ]:
# %load sol_model_mnist

model = nn.Sequential(
    # Ecrire ici
)

print(model)

### Test du réseau avant entrainement

In [ ]:
fig = plt.figure(figsize=(16,9))
image_indices = torch.randint(0, len(val_dataset), (5,))  # On choisit 5 indices au hasard
for i in range(0, 5):
    j = image_indices[i].item()
    data, true_label = val_dataset[j]
    inputs = data.view(1, 784)  # On transforme la matrice 28x28 en vecteur de taille 784
    outputs = model(inputs)
    predicted_label = outputs.argmax(1).item()
    
    fig.add_subplot(1, 5, i+1)
    plt.imshow(data.squeeze(), cmap="gray")
    plt.xlabel(predicted_label, fontsize=20)

### Entrainement du modèle

Ici, une seule époque suffit pour avoir des résultats corrects.

In [ ]:
batch_size = 100
nb_epochs = 1
lr = 0.5

dataloader = DataLoader(train_dataset, batch_size, shuffle=True)
optimizer = optim.SGD(model.parameters(), lr)
loss_function = nn.CrossEntropyLoss()

for epoch in range(nb_epochs):
    print('Epoque %d' % epoch)
    for inputs, targets in dataloader:
        inputs = inputs.view(batch_size, 28*28) # On transforme la matrice 28x28 en vecteur de taille 784
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = loss_function(outputs, targets)
        loss.backward()
        optimizer.step()
print("-- Fin --")

### Validation du modèle

In [ ]:
dataloader = DataLoader(val_dataset, batch_size)

acc = 0
for inputs, targets in dataloader:
    inputs = inputs.view(-1, 28*28)
    outputs = model(inputs)
    predictions = outputs.argmax(1)
    acc += (predictions == targets).sum()

acc = acc.float() / len(dataloader.dataset)
print("Précision: {:.2%}".format(acc))

### Test du réseau après entrainement

In [ ]:
fig = plt.figure(figsize=(16,9))
image_indices = torch.randint(0, len(val_dataset), (5,))
for i in range(0, 5):
    j = image_indices[i].item()
    data, true_label = val_dataset[j]
    inputs = data.view(1, 784)
    outputs = model(inputs)
    predicted_label = outputs.argmax(1).item()
    data = data.squeeze()
    fig.add_subplot(1, 5, i+1)
    plt.imshow(data, cmap="gray")
    plt.xlabel(predicted_label, fontsize=20)

# >>> [Cliquez ici pour aller à la suite du TP](Using-a-pretrained-CNN.ipynb) <<<